In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from networkx.algorithms.community.quality import modularity
import warnings
import os

warnings.filterwarnings("ignore", category=UserWarning)

# ---------------------------
# Graph Colouring Algorithms
# ---------------------------

def greedy_colour_cluster(G):
    coloring = {}
    for node in sorted(G.nodes(), key=lambda n: G.degree(n), reverse=True):
        used = {coloring[nbr] for nbr in G.neighbors(node) if nbr in coloring}
        color = 0
        while color in used:
            color += 1
        coloring[node] = color
    labels = np.array([coloring[node] for node in G.nodes()])
    return labels, coloring

def dsatur_colour_cluster(G):
    sat_deg = {node: 0 for node in G.nodes()}
    neighbor_colors = {node: set() for node in G.nodes()}
    coloring = {}

    first = max(G.nodes(), key=lambda n: G.degree(n))
    coloring[first] = 0
    for nbr in G.neighbors(first):
        neighbor_colors[nbr].add(0)
        sat_deg[nbr] = 1

    uncolored = set(G.nodes()) - {first}
    while uncolored:
        u = max(uncolored, key=lambda n: (sat_deg[n], G.degree(n)))
        used = neighbor_colors[u]
        color = 0
        while color in used:
            color += 1
        coloring[u] = color
        uncolored.remove(u)
        for nbr in G.neighbors(u):
            if nbr in uncolored:
                neighbor_colors[nbr].add(color)
                sat_deg[nbr] = len(neighbor_colors[nbr])

    labels = np.array([coloring[node] for node in G.nodes()])
    return labels, coloring

def equitable_colour_cluster(G):
    labels, coloring = dsatur_colour_cluster(G)
    return labels, coloring

def distance_k_colour_cluster(G, k):
    Gk = nx.power(G, k)
    return greedy_colour_cluster(Gk)

# ---------------------------
# Evaluation Functions
# ---------------------------

def evaluate_clustering(G, labels):
    communities = {}
    for node, label in zip(G.nodes(), labels):
        communities.setdefault(label, set()).add(node)
    community_list = list(communities.values())
    mod_score = modularity(G, community_list)
    sil_score = -1
    try:
        adj_matrix = nx.to_numpy_array(G)
        if len(set(labels)) > 1:
            sil_score = silhouette_score(adj_matrix, labels, metric='euclidean')
    except:
        pass
    return mod_score, sil_score, len(set(labels))

# ---------------------------
# Visualisation Function
# ---------------------------

def draw_coloured_graph(G, labels, title, save_path):
    pos = nx.spring_layout(G, seed=42)
    cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)
    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(G, pos, node_color=labels, cmap=cmap, node_size=50)
    nx.draw_networkx_edges(G, pos, alpha=0.3)
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# ---------------------------
# Synthetic Graph Generation
# ---------------------------

def generate_bipartite_graph(n1, n2, p):
    top = range(n1)
    bottom = range(n1, n1 + n2)
    edges = [(u, v) for u in top for v in bottom if np.random.rand() < p]
    B = nx.Graph()
    B.add_nodes_from(top, bipartite=0)
    B.add_nodes_from(bottom, bipartite=1)
    B.add_edges_from(edges)
    return B

def generate_synthetic_graphs():
    graphs = {
        "Erdos-Renyi": nx.erdos_renyi_graph(100, 0.05),
        "StochasticBlockModel": nx.stochastic_block_model(
            [30, 30, 40],
            [[0.1, 0.02, 0.01], [0.02, 0.1, 0.02], [0.01, 0.02, 0.1]]
        ),
        "Barabasi-Albert": nx.barabasi_albert_graph(100, 3),
        "Bipartite": generate_bipartite_graph(30, 30, 0.1)
    }
    return graphs

# ---------------------------
# Experiment Runner
# ---------------------------

def run_all_experiments():
    graphs = generate_synthetic_graphs()
    os.makedirs("output", exist_ok=True)

    for name, G in graphs.items():
        print(f"\n=== {name} Graph ===")
        for method_name, method in {
            "Greedy": greedy_colour_cluster,
            "DSATUR": dsatur_colour_cluster,
            "Equitable": equitable_colour_cluster,
            "Distance-2": lambda g: distance_k_colour_cluster(g, 2)
        }.items():
            labels, _ = method(G)
            mod, sil, chrom = evaluate_clustering(G, labels)
            print(f"{method_name}: Modularity={mod:.3f}, Silhouette={sil:.3f}, Colours={chrom}")
            title = f"{name} - {method_name}"
            filename = f"output/{name}_{method_name}.png".replace(" ", "_")
            draw_coloured_graph(G, labels, title, filename)

# ---------------------------
# Main Execution
# ---------------------------

if __name__ == "__main__":
    print("Running generalised graph colouring experiments with visualisations...")
    run_all_experiments()



Running generalised graph colouring experiments with visualisations...

=== Erdos-Renyi Graph ===
Greedy: Modularity=-0.270, Silhouette=-0.008, Colours=4
DSATUR: Modularity=-0.283, Silhouette=-0.021, Colours=4


/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)
/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)


Equitable: Modularity=-0.283, Silhouette=-0.021, Colours=4
Distance-2: Modularity=-0.076, Silhouette=-0.140, Colours=15


/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)
/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)



=== StochasticBlockModel Graph ===
Greedy: Modularity=-0.301, Silhouette=-0.009, Colours=4
DSATUR: Modularity=-0.304, Silhouette=-0.014, Colours=4


...

/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)
/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)


Distance-2: Modularity=-0.053, Silhouette=-0.228, Colours=25

=== Bipartite Graph ===
Greedy: Modularity=-0.500, Silhouette=0.073, Colours=2


/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)
/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)
/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot

DSATUR: Modularity=-0.500, Silhouette=0.073, Colours=2
Equitable: Modularity=-0.500, Silhouette=0.073, Colours=2
Distance-2: Modularity=-0.124, Silhouette=-0.066, Colours=9


/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)
/var/folders/t5/yks90q315p998cfvk7kvyfcw0000gn/T/ipykernel_14505/1118710396.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', np.max(labels) + 1)
